# Setup

In [1]:
!pip install -q anthropic IPython pdf2image PyMuPDF

Ten kod instaluje cztery pakiety Pythona:

1. **anthropic** - biblioteka kliencka umożliwiająca integrację z modelami językowymi Claude od firmy Anthropic. Pozwala na komunikację z API Anthropic, wysyłanie zapytań do modeli Claude i odbieranie odpowiedzi.

2. **IPython** - rozszerzony, interaktywny interpreter Pythona oferujący wzbogacone środowisko programistyczne z funkcjami takimi jak podświetlanie składni, uzupełnianie kodu i wyświetlanie dokumentacji. Jest kluczowym komponentem Jupyter Notebook.

3. **pdf2image** - narzędzie do konwertowania plików PDF na obrazy. Umożliwia przekształcanie stron PDF na formaty obrazów takie jak PNG, JPEG itp., co jest przydatne przy analizie wizualnej dokumentów lub przetwarzaniu zawartości PDF.

4. **PyMuPDF** - zaawansowana biblioteka do pracy z dokumentami PDF (a także XPS, EPUB i innymi). Oferuje szeroki zakres funkcji do odczytywania, modyfikowania i analizowania plików PDF, włączając w to ekstrakcję tekstu, obrazów, manipulację stronami i wiele innych.


In [ ]:
import base64
from anthropic import Anthropic
from PIL import Image
import io
import fitz # PyMuPDF
from tqdm import tqdm
from google.colab import userdata

`import base64` - importuje moduł do kodowania i dekodowania danych w formacie Base64, często używanym do przekształcania danych binarnych (jak obrazy) na tekst, który można bezpiecznie przesyłać przez Internet.

`from anthropic import Anthropic` - importuje klasę Anthropic z pakietu anthropic, która służy do komunikacji z API Anthropic, umożliwiając dostęp do modeli językowych Claude.

`from PIL import Image` - importuje klasę Image z biblioteki PIL (Python Imaging Library, znana także jako Pillow), która oferuje zaawansowane funkcje do przetwarzania obrazów.

`import io` - importuje moduł io, który dostarcza narzędzia do obsługi strumieni danych (wejścia/wyjścia), szczególnie przydatny przy konwersji między różnymi formatami danych.

`import fitz` - importuje bibliotekę fitz, która jest częścią pakietu PyMuPDF, służącą do manipulacji dokumentami PDF.

`import re` - importuje moduł do pracy z wyrażeniami regularnymi, co umożliwia zaawansowane wyszukiwanie i manipulację tekstem.

`from tqdm import tqdm` - importuje funkcję tqdm, która tworzy paski postępu w terminalu lub notatniku, co jest przydatne przy monitorowaniu długotrwałych operacji.

In [3]:
class CFG:
    model = "claude-3-opus-20240229"
    max_tokens = 2048
    deck_path = "/content/twilio_q4_2023.pdf"

In [4]:
client = Anthropic(api_key = userdata.get('claude'))

# Funkcje

In [5]:
def get_completion(messages):

    response = client.messages.create(
        model = CFG.model,
        max_tokens = CFG.max_tokens,
        temperature=0,
        messages=messages
    )

    return response.content[0].text

In [ ]:
def pdf_to_base64_pngs(pdf_path, pages_to_convert=10, max_size=(1024, 1024)):
    doc = fitz.open(pdf_path)
    base64_encoded_pngs = []

    for page_num in range(min(pages_to_convert, doc.page_count)):
        page = doc.load_page(page_num)
        pix = page.get_pixmap(matrix=fitz.Matrix(300 / 72, 300 / 72))
        
        # Konwersja pixmapy do obrazu PIL bez zapisu na dysk
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        # Skalowanie, jeśli obraz przekracza maksymalny rozmiar
        if img.size[0] > max_size[0] or img.size[1] > max_size[1]:
            img.thumbnail(max_size, Image.Resampling.LANCZOS)
        
        # Konwersja obrazu do Base64
        image_data = io.BytesIO()
        img.save(image_data, format="PNG", optimize=True)
        image_data.seek(0)
        base64_encoded = base64.b64encode(image_data.getvalue()).decode("utf-8")
        base64_encoded_pngs.append(base64_encoded)
    
    doc.close()
    return base64_encoded_pngs

**Opis funkcji `pdf_to_base64_pngs`**

**Zadanie funkcji**  
Funkcja konwertuje pierwsze `pages_to_convert` stron PDF na obrazy PNG i zwraca je w formacie Base64. Wszystkie operacje odbywają się w pamięci.

**Działanie funkcji**  

1. **Otwiera plik PDF** za pomocą `PyMuPDF`.  
2. **Iteruje przez strony** (maksymalnie `pages_to_convert`).  
3. **Renderuje stronę jako obraz** w wysokiej rozdzielczości (300 DPI).  
4. **Konwertuje `pixmap` na obraz PIL**, bez zapisywania na dysk.  
5. **Skaluje obraz**, jeśli przekracza `max_size` (domyślnie `1024x1024`).  
6. **Konwertuje obraz na Base64**, zapisując go najpierw w buforze pamięci (`io.BytesIO`).  
7. **Zamyka plik PDF** i zwraca listę Base64 zakodowanych obrazów.  

In [ ]:
def build_previous_slides_prompt(previous_slide_narratives):
    prompt = '\n'.join([f"<slide_narration id={index+1}>\n{narrative}\n</slide_narration>" for index, narrative in enumerate(previous_slide_narratives)])
    return prompt

In [ ]:
def build_slides_narration_prompt(previous_slide_narratives):
    if len(previous_slide_narratives) == 0:
        prompt = """You are the Twilio CFO, narrating your Q4 2023 earnings presentation.
                You are currently on slide 1, shown in the image.
                Please narrate this page from Twilio's Q4 2023 Earnings Presentation as if you were the presenter.
                Do not talk about any things, especially acronyms, if you are not exactly sure you know what they mean.
                Do not discuss anything not explicitly seen on this slide as there are more slides \
                to narrate later that will likely cover that material.
                Do not leave any details un-narrated as some of your viewers are vision-impaired,\
                so if you don't narrate every number they won't know the number. """
    else:
        prompt = f"""You are the Twilio CFO, narrating your Q4 2023 earnings presentation. So far, here is your narration from previous slides:
        <previous_slide_narrations>
        {build_previous_slides_prompt(previous_slide_narratives)}
        </previous_slide_narrations>
        You are currently on slide {len(previous_slide_narratives) + 1}, shown in the image.
        Please narrate this page from Twilio's Q4 2023 Earnings Presentation as if you were the presenter,\
        accounting for what you have already said on previous slides. Do not talk about any things, especially acronyms,
        if you are not exactly sure you know what they mean. Do not discuss anything not explicitly seen on this slide \
        as there are more slides to narrate later that will likely cover that material.\
        Do not leave any details un-narrated as some of your viewers are vision-impaired,\
        so if you don't narrate every number they won't know the number.
        Use excruciating detail. """

    return prompt

# Narracja krok po kroku

In [ ]:
encoded_pngs = pdf_to_base64_pngs(CFG.deck_path)

In [ ]:
content = [
    {
        "type": "image",
        "source": {"type": "base64", "media_type": "image/png", "data": encp},
    }
    for encp in encoded_pngs
]

question = "What was Twilio y/y revenue growth for fiscal year 2023?"
content.append({"type": "text", "text": question})

messages = [{"role": "user", "content": content}]

print(get_completion(messages))

According to the Total Company Results Highlights table, Twilio's Y/Y Revenue Growth for fiscal year 2023 was 9%.


In [11]:
previous_slide_narratives = []

for i, encoded_png in tqdm(enumerate(encoded_pngs)):
    messages = [
        {
            "role": 'user',
            "content": [
            {"type": "image", "source": {"type": "base64", "media_type": "image/png", "data": encoded_png}},
            {"type": "text", "text": build_slides_narration_prompt(previous_slide_narratives)}
        ]
        }
    ]
    completion = get_completion(messages)

    previous_slide_narratives.append(completion)

slide_narration = build_previous_slides_prompt(previous_slide_narratives)

10it [03:38, 21.87s/it]


In [ ]:
questions = [
    "What percentage of q4 total revenue was the Segment business line?",
    "Has the rate of growth of quarterly revenue been increasing or decreasing? Give just an answer.",
    "What was acquisition revenue for the year ended december 31, 2023 (including negative revenues)?",
]


for index, question in enumerate(questions):
    prompt = f"""You are an expert financial analyst analyzing a transcript of Twilio's earnings call.

            Here is the transcript:

            <transcript>

            {slide_narration}

            </transcript>



            Please answer the following question:

            <question>

            {question}

            </question>"""

    messages = [{"role": "user", "content": [{"type": "text", "text": prompt}]}]

    print(f"\n----------Question {index + 1}----------")
    print(get_completion(messages))


----------Question 1----------
To calculate the percentage of Q4 total revenue that the Segment business line represented, I'll find the Q4 2023 revenue numbers for Segment and Total revenue.

From slide 6:
"In Q4 2023, Segment Revenue grew to $75 million dollars."

From slide 3:
"Total Revenue is $1,190 million for Q4 2023."

Percentage calculation:
Segment Revenue / Total Revenue * 100
= $75 million / $1,190 million * 100
= 0.0630 * 100
= 6.30%

Therefore, in Q4 2023, Twilio's Segment business line revenue of $75 million represented approximately 6.30% of the company's total revenue of $1,190 million.

----------Question 2----------
Based on the information provided in the earnings call transcript, the rate of growth of Twilio's quarterly revenue has been decreasing over the past two years from Q4 2021 to Q4 2023.

----------Question 3----------
To determine the acquisition revenue for the year ended December 31, 2023, I'll subtract the organic revenue from the total revenue for fis